### Data-Generator

In [0]:
from pyspark.sql import Row
from pyspark.sql.functions import col, to_date
from datetime import datetime, timedelta
import random

# Datos base
clientes = ['Juan', 'María', 'Carlos', 'Ana', 'Lucía', "Stephanie", "Felipe", "Fernando", "Diego", "Humberto", "Marcelo", "Roberto", "Pedro"]
productos = ['Zapato', 'Remera', 'Pantalón', 'Gorra', 'Bufanda', "Medias", "Saco", "Smoking", "Zapatillas", "Pijama"]

# Intentar leer la tabla para obtener el último ID
try:
    df_existente = spark.table("workspace.default.ventas_delta")
    ultimo_id = df_existente.agg({"venta_id": "max"}).collect()[0][0]
    if ultimo_id is None:
        ultimo_id = 0
except Exception:
    # Si la tabla no existe
    ultimo_id = 0

# Generar nuevas filas
fecha_inicio = datetime(2025, 1, 1)
filas = []
for i in range(100):
    fila = Row(
        venta_id = ultimo_id + i + 1,
        cliente = random.choice(clientes),
        producto = random.choice(productos),
        cantidad = random.randint(1, 5),
        precio_unitario = random.randint(100, 1000),
        fecha_venta = fecha_inicio + timedelta(days=random.randint(0, 180))
    )
    filas.append(fila)

df_spark = spark.createDataFrame(filas)

# Formatear fecha
df_spark = df_spark.withColumn("fecha_venta", to_date(col("fecha_venta")))

# Guardar en Delta (append si ya existe)
df_spark.write.format("delta").mode("append").saveAsTable("workspace.default.ventas_bronze")

print(f"Se agregaron {len(filas)} filas nuevas. Último ID ahora: {ultimo_id + len(filas)}")
